In [2]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

c:\Users\jaesc2\GitHub\skforecast


In [1]:
import numpy as np

In [20]:
def coverage(y: np.ndarray, lower_bound: np.ndarray, upper_bound: np.ndarray) -> float:
    """
    Calculate coverage of a given interval
    """

    if not isinstance(y, np.ndarray) or y.ndim != 1:
        raise TypeError("`y` must be a 1D numpy array.")
    
    if not isinstance(lower_bound, np.ndarray) or lower_bound.ndim != 1:
        raise TypeError("`lower_bound` must be a 1D numpy array.")
    
    if not isinstance(upper_bound, np.ndarray) or upper_bound.ndim != 1:
        raise TypeError("`upper_bound` must be a 1D numpy array.")
    
    if y.shape != lower_bound.shape or y.shape != upper_bound.shape:
        raise TypeError("`y`, `lower_bound`, and `upper_bound` must have the same shape.")
    
    coverage = np.mean(np.logical_and(y >= lower_bound, y <= upper_bound))

    return coverage

In [23]:
def test_coverage_raise_error_when_no_valid_inputs():

    y = np.array([1, 2])
    lower_bound = np.array([1, 2])
    upper_bound = np.array([2, 3])

    msg = "`y`, must be a 1D numpy array."
    with pytest.raises(TypeError):
        coverage(y='invalid value', lower_bound, upper_bound)

    msg = "`lower_bound`, must be a 1D numpy array."
    with pytest.raises(TypeError):
        coverage(y, lower_bound='invalid value', upper_bound)

    msg = "`upper_bound`, must be a 1D numpy array."
    with pytest.raises(TypeError):
        coverage(y, lower_bound, upper_bound='invalid value')

    msg = "`y`, `lower_bound`, and `upper_bound` must have the same shape."
    with pytest.raises(TypeError):
        coverage(y, lower_bound=np.array([1, 2, 3]), upper_bound)
    with pytest.raises(TypeError):
        coverage(y, lower_bound, upper_bound=np.array([1, 2, 3]))
    with pytest.raises(TypeError):
        coverage(y=np.array([1, 2, 3]), lower_bound, upper_bound)

    

0.9

In [25]:
def test_coverage_output():
    """
    Test the output of the coverage function when 10 out of 100 values are outside
    the upper and lower bounds.
    """
    lower_bound = np.random.normal(10, 2, 100)
    upper_bound = lower_bound * 2
    y = (lower_bound + upper_bound) / 2
    y[[10, 12, 15, 20, 25, 30, 35, 40, 45, 50]] = (
        y[[10, 12, 15, 20, 25, 30, 35, 40, 45, 50]] * 10
    )
    results = coverage(y, lower_bound, upper_bound)
    expected = 0.9

    assert results == expected

test_coverage_output()